In [1]:
# Import Library
import pandas as pd
import glob
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random
from sklearn import svm
from sklearn.model_selection import cross_validate

# 📂DATA : 2 Classes

In [33]:
fold = 3

In [34]:
path = "/home/kannika/code/Rheology2023/Rheology_Blood/DataBlood_Viscosity_TrainML_3Fold_EMclass.csv"
data_feature = pd.read_csv(path)
print(data_feature.shape)
print("-"*100)
print(f"All Fold : {set(data_feature.fold)}")
## Split Train data Set
feature_train = data_feature[data_feature["fold"]!=fold].reset_index(drop=True)
print(f"Train Set : Fold ==> {set(feature_train.fold)}")
print("Train = ", feature_train.shape)
## Split Valid data Set
feature_test = data_feature[data_feature["fold"]==fold].reset_index(drop=True)
print(f"Valiadtion Set : Fold ==> {set(feature_test.fold)}")
print("Validation = ", feature_test.shape)
## Print DataFrame
feature_train.head()

(33, 11)
----------------------------------------------------------------------------------------------------
All Fold : {1, 2, 3}
Train Set : Fold ==> {1, 2}
Train =  (22, 11)
Valiadtion Set : Fold ==> {3}
Validation =  (11, 11)


Unnamed: 0  Unnamed: 0.1  Code classes        subclass  classes_binary  \
0           0            42  HN44      HN     Splenectomy             1.0   
1           1            17  HN21      HN  No_Splenectomy             1.0   
2           2             8  HN13      HN     Splenectomy             1.0   
3           3            43  HN29      HN  No_Splenectomy             1.0   
4           4            50  HN39      HN     Splenectomy             1.0   

    MCV   MCH   Hb typeBEvsBM  fold  
0  61.7  19.7  7.0          E     1  
1  64.8  19.7  6.5          E     1  
2  72.6  23.4  5.8          E     1  
3  64.2  19.1  6.6          E     1  
4  76.4  22.5  6.4          E     1

In [35]:
X_train = feature_train[['MCV','MCH','Hb']]
y_train = feature_train["typeBEvsBM"]
print(X_train.shape)
print(y_train.shape)
print(y_train[0])

(22, 3)
(22,)
E


In [36]:
print(len(list(set(y_train))))
print(set(y_train))

2
{'M', 'E'}


## 🩸 Parameter:  SVM

In [37]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report,confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [38]:
Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.001, 0.01, 0.1, 1]
kernels = ['linear', 'rbf']

## Find optimiz paremeter 
from sklearn import svm
svm = svm.SVC(random_state = 1)
param_grid = {'C': Cs, 'gamma' : gammas, 'kernel' : kernels}
grid_search = GridSearchCV(svm, param_grid, cv=10, verbose=1, n_jobs=-1) ##cv=10 == 10-fold validation 
SVM = grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 40 candidates, totalling 400 fits


In [39]:
# view the results as a pandas DataFrame
best_svm_df = pd.DataFrame(SVM.cv_results_)[['rank_test_score', 'mean_test_score', 'std_test_score', 'params']]
best_svm_df = best_svm_df.sort_values(by='rank_test_score', ascending=True)
best_svm_df = best_svm_df.reset_index(drop=True)
best_svm_df.head()

rank_test_score  mean_test_score  std_test_score  \
0                1         0.866667        0.208167   
1                1         0.866667        0.208167   
2                3         0.766667        0.238048   
3                4         0.733333        0.226078   
4                4         0.733333        0.226078   

                                             params  
0          {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}  
1         {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}  
2           {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}  
3  {'C': 0.001, 'gamma': 0.001, 'kernel': 'linear'}  
4       {'C': 10, 'gamma': 0.1, 'kernel': 'linear'}

In [40]:
# examine the first result
print("**examine the first result","\n")

print(SVM.cv_results_['params'][0])
print(SVM.cv_results_['mean_test_score'][0])

# print the array of mean scores only
print("\n","**print the array of mean scores only","\n")

grid_mean_scores = SVM.cv_results_['mean_test_score']
print(grid_mean_scores)

# examine the best model
print("\n","**examine the best model","\n")

print(SVM.best_score_)
print(SVM.best_params_)
print(SVM.best_estimator_)

**examine the first result 

{'C': 0.001, 'gamma': 0.001, 'kernel': 'linear'}
0.7333333333333333

 **print the array of mean scores only 

[0.73333333 0.73333333 0.73333333 0.73333333 0.73333333 0.73333333
 0.73333333 0.73333333 0.73333333 0.73333333 0.73333333 0.73333333
 0.73333333 0.73333333 0.73333333 0.73333333 0.68333333 0.73333333
 0.68333333 0.73333333 0.68333333 0.73333333 0.68333333 0.73333333
 0.68333333 0.73333333 0.68333333 0.73333333 0.68333333 0.76666667
 0.68333333 0.73333333 0.73333333 0.73333333 0.73333333 0.86666667
 0.73333333 0.86666667 0.73333333 0.73333333]

 **examine the best model 

0.8666666666666666
{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
SVC(C=10, gamma=0.01, random_state=1)


In [41]:
#Print the tured parameters and score
print("Tuned Decision Tree Parameters: {}".format(SVM.best_params_))
print("Best score is {}".format(SVM.best_score_))

Tuned Decision Tree Parameters: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
Best score is 0.8666666666666666


In [43]:
##print(bestF)
best_params_ = SVM.best_params_
C_ = best_params_['C']  ##<--- best_params = Cs
gamma_ = best_params_['gamma'] ##<--- best_params = gammas
kernel_ = best_params_['kernel'] ##<--- best_params = kernels

> ## 🚀 Fit Model and setting parameters

In [44]:
from sklearn import svm

SVM = svm.SVC(random_state = 1, kernel=kernel_, C=C_, gamma=gamma_, probability=True ) #เปลี่ยนตาม parameter
modelSVM = SVM.fit(X_train, y_train) # train SVM model
modelSVM

SVC(C=10, gamma=0.01, probability=True, random_state=1)

In [45]:
### Testing Check Parameter
params = modelSVM.get_params()
print(params)

{'C': 10, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.01, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': 1, 'shrinking': True, 'tol': 0.001, 'verbose': False}


> ## #️⃣ Save model

In [46]:
nameclass = "typeBEvsBM"

In [47]:
#Save Model
import os
import pickle
import imageio

save_pathimg = f'/media/tohn/HDD/rheology2023/ML_Model/Blood_Viscosity/{nameclass}_Classes/fold{fold}'
##**Mkdir Directory 
os.makedirs(save_pathimg, exist_ok=True)     
filename = f"SVM_Model_{nameclass}_Class_fold{fold}.pkl"
Model2Save =  f"{save_pathimg}/{filename}"
print(f"[INFO]: Save Nodel as : [{Model2Save}]")

with open(Model2Save, 'wb') as file:
    pickle.dump(modelSVM, file)

[INFO]: Save Nodel as : [/media/tohn/HDD/rheology2023/ML_Model/Blood_Viscosity/typeBEvsBM_Classes/fold3/SVM_Model_typeBEvsBM_Class_fold3.pkl]
